In [1]:
# closest genes to diff peaks
# closest genes to merged peaks of each cond at each time
# merged peaks overlap

In [2]:
import os
import numpy as np
import pandas as pd
import subprocess as sp
from importlib import reload 
import pybedtools as pbt
pbt.set_bedtools_path("/localenv/rabbani/anaconda/miniconda3/envs/matplotlib/bin/")
pbt = reload(pbt)

In [3]:
diff_peaks = pd.read_csv("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                         "deseq_timecourse_ressig_lrt.tsv", sep = "\t")
diff_peaks_coord = pd.DataFrame()
diff_peaks_coord[['chr', 'start', 'end']] = diff_peaks['gene'].str.split('_', 2, expand=True)
diff_peaks_coord

,chr,start,end
0,7,49550979,49551414
1,6,87745250,87745665
2,8,95695285,95695489
3,8,126844604,126845030
4,1,178528166,178528317
...,...,...,...
319,7,19327056,19327533
320,11,50197277,50197837
321,4,43043900,43044252
322,13,41331582,41332217


In [4]:
diff_bed = pbt.BedTool.from_dataframe(diff_peaks_coord)
diff_bed = diff_bed.sort()

In [5]:
genes = pbt.BedTool("/data/repository/organisms/GRCm38_ensembl/gencode/m9/genes.gtf")
genes = genes.sort()
closest_genes = diff_bed.closest(genes, d=True)

In [6]:
closest_genes.saveas("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                     "closest_genes_2_diff_peaks.bed")

<BedTool(/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/closest_genes_2_diff_peaks.bed)>

In [7]:
# extract gene_name
df = pd.read_csv("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                     "closest_genes_2_diff_peaks.bed", sep = "\t", header = None, usecols = [0,1,2,5,11,12])
df = df.loc[df[5] == 'gene']
df

,0,1,2,5,11,12
0,1,24612912,24613318,gene,"gene_id ""ENSMUSG00000101939.1""; gene_type ""unp...",0
5,1,24612912,24613318,gene,"gene_id ""ENSMUSG00000101111.1""; gene_type ""unp...",0
6,1,36287740,36288193,gene,"gene_id ""ENSMUSG00000047180.8""; gene_type ""pro...",13062
11,1,37676066,37676221,gene,"gene_id ""ENSMUSG00000026090.16""; gene_type ""pr...",0
13,1,64807127,64807466,gene,"gene_id ""ENSMUSG00000051344.13""; gene_type ""pr...",0
...,...,...,...,...,...,...
1867,9,110681897,110682075,gene,"gene_id ""ENSMUSG00000019659.8""; gene_type ""pro...",0
1874,9,119569219,119569686,gene,"gene_id ""ENSMUSG00000032511.17""; gene_type ""pr...",0
1877,9,121404298,121404490,gene,"gene_id ""ENSMUSG00000032536.9""; gene_type ""pro...",0
1880,9,123461818,123462260,gene,"gene_id ""ENSMUSG00000035202.7""; gene_type ""pro...",0


In [8]:
df1 = df[11].str.split(";")
print(len(df1))
df1

350


0       [gene_id "ENSMUSG00000101939.1",  gene_type "u...
5       [gene_id "ENSMUSG00000101111.1",  gene_type "u...
6       [gene_id "ENSMUSG00000047180.8",  gene_type "p...
11      [gene_id "ENSMUSG00000026090.16",  gene_type "...
13      [gene_id "ENSMUSG00000051344.13",  gene_type "...
                              ...                        
1867    [gene_id "ENSMUSG00000019659.8",  gene_type "p...
1874    [gene_id "ENSMUSG00000032511.17",  gene_type "...
1877    [gene_id "ENSMUSG00000032536.9",  gene_type "p...
1880    [gene_id "ENSMUSG00000035202.7",  gene_type "p...
1883    [gene_id "ENSMUSG00000025240.8",  gene_type "p...
Name: 11, Length: 350, dtype: object

In [9]:
gene_names = os.path.join("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                          "closest_genenames.tsv")

with open(gene_names, "w") as names:
    for gene in df1:
        this_id = gene[0].split("\"")[1]
        coord = df.loc[df[11].str.contains(this_id), [0,1,2]].values[0]
        this_name = gene[3].split("\"")[1]
        wo_version = this_id.split(".")[0]
        names.write(str(coord[0])+"\t"+str(coord[1])+"\t"+str(coord[2])+"\t"+this_id+"\t"+this_name+"\t"+wo_version+"\n")

In [10]:
diff_bed.saveas("/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/"\
                "diff_peaks.bed")

<BedTool(/data/manke/group/rabbani/atac_project2078/timecourse_diff_analysis/diff_peaks.bed)>

In [ ]:
# merged peaks overlap